In [1]:
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import numpy as np
df_train_labels_original = pd.read_csv('train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id').apply(lambda x: x-1)
df_train_values_original = pd.read_csv('train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'int64',
    'geo_level_3_id':'int64', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id').drop(columns=['geo_level_3_id'])


pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
df = df_train_values_original.join(df_train_labels_original,how="inner")
df

,geo_level_1_id,geo_level_2_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,2,30,6,5,t,r,n,f,...,0,0,0,0,0,0,0,0,0,2
28830,8,900,2,10,8,7,o,r,n,x,...,0,0,0,0,0,0,0,0,0,1
94947,21,363,2,10,5,5,t,r,n,f,...,0,0,0,0,0,0,0,0,0,2
590882,22,418,2,10,6,5,t,r,n,f,...,0,0,0,0,0,0,0,0,0,1
201944,11,131,3,30,8,9,t,r,n,f,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,25,1335,1,55,6,3,n,r,n,f,...,0,0,0,0,0,0,0,0,0,1
669485,17,715,2,0,6,5,t,r,n,f,...,0,0,0,0,0,0,0,0,0,2
602512,17,51,3,55,6,7,t,r,q,f,...,0,0,0,0,0,0,0,0,0,2


In [3]:
def mean_encode(dataframe, column_name):
    new_column_names = {
        0: column_name+'_0',
        1: column_name+'_1',
        2: column_name+'_2',
    }
    cross = pd.crosstab(dataframe[column_name], dataframe['damage_grade']).rename(columns=new_column_names)
    prob = cross.divide(cross.apply('sum',axis=1),axis=0).reset_index()
    return dataframe.reset_index().merge(prob,on=column_name).set_index('building_id').drop(columns=[column_name])

In [4]:
def one_hot_encode_data(dataframe, column_name):
    dummies = pd.get_dummies(dataframe[column_name])
    rename_columns = {}
    for column in dummies.columns.values:
        rename_columns[column] = column_name + '_' + column
    return dataframe.drop(columns=[column_name]).join(dummies.rename(columns=rename_columns))

In [5]:
#df = mean_encode(df, 'geo_level_2_id')
df = one_hot_encode_data(df,'land_surface_condition')
df = one_hot_encode_data(df,'foundation_type')
df = one_hot_encode_data(df,'roof_type')
df = one_hot_encode_data(df,'ground_floor_type')
df = one_hot_encode_data(df,'other_floor_type')
df = one_hot_encode_data(df,'position')
df = one_hot_encode_data(df,'plan_configuration')
df = one_hot_encode_data(df,'legal_ownership_status')
df = one_hot_encode_data(df,'geo_level_1_id')
df

,geo_level_2_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,...,geo_level_1_id_28,geo_level_1_id_29,geo_level_1_id_3,geo_level_1_id_30,geo_level_1_id_4,geo_level_1_id_5,geo_level_1_id_6,geo_level_1_id_7,geo_level_1_id_8,geo_level_1_id_9
building_id,,,,,,,,,,,,,,,,,,,,,
802906,487,2,30,6,5,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
28830,900,2,10,8,7,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
94947,363,2,10,5,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
590882,418,2,10,6,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201944,131,3,30,8,9,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,1335,1,55,6,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
669485,715,2,0,6,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
602512,51,3,55,6,7,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
train_df, target = (df.drop(columns=['damage_grade'])[:int(len(df)/2)], df['damage_grade'][:int(len(df)/2)])
test_df, test_target = (df.drop(columns=['damage_grade'])[int(len(df)/2):], df['damage_grade'][int(len(df)/2):])
train_df

,geo_level_2_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,...,geo_level_1_id_28,geo_level_1_id_29,geo_level_1_id_3,geo_level_1_id_30,geo_level_1_id_4,geo_level_1_id_5,geo_level_1_id_6,geo_level_1_id_7,geo_level_1_id_8,geo_level_1_id_9
building_id,,,,,,,,,,,,,,,,,,,,,
802906,487,2,30,6,5,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
28830,900,2,10,8,7,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
94947,363,2,10,5,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
590882,418,2,10,6,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201944,131,3,30,8,9,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168990,1132,2,5,7,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
966340,36,1,5,7,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
842927,755,2,35,6,7,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [7]:
test_df

,geo_level_2_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,...,geo_level_1_id_28,geo_level_1_id_29,geo_level_1_id_3,geo_level_1_id_30,geo_level_1_id_4,geo_level_1_id_5,geo_level_1_id_6,geo_level_1_id_7,geo_level_1_id_8,geo_level_1_id_9
building_id,,,,,,,,,,,,,,,,,,,,,
666622,1023,1,10,10,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
194057,463,2,5,8,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
736347,1131,1,5,16,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69992,1030,1,5,4,4,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
133103,1115,2,15,10,5,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,1335,1,55,6,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
669485,715,2,0,6,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
602512,51,3,55,6,7,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=4, shuffle=True).split(train_df, target)
generic_prob = pd.crosstab(train_df['geo_level_2_id'], target).applymap(lambda x: 1/3)
train_df_after = pd.DataFrame()
average_damage = pd.get_dummies(target).mean().values

for training_index, validation_index in folds:
    cross = pd.crosstab(train_df.iloc[training_index]['geo_level_2_id'], target.iloc[training_index])
    prob = cross.divide(cross.apply('sum',axis=1),axis=0).reset_index()
    result = train_df.iloc[validation_index].reset_index().merge(prob,on='geo_level_2_id',how='left')
    result[0] = result[0].fillna(average_damage[0])
    result[1] = result[1].fillna(average_damage[1])
    result[2] = result[2].fillna(average_damage[2])
    result = result.set_index('building_id')
    train_df_after = train_df_after.append(result)

alfa = 10
n_rows = train_df.groupby('geo_level_2_id').count()['height_percentage'].rename('n_rows')
train_df_after = train_df_after.reset_index().merge(n_rows,on='geo_level_2_id',how='left').set_index('building_id')
train_df_after[0] = train_df_after[0].multiply(train_df_after['n_rows'])
train_df_after[1] = train_df_after[1].multiply(train_df_after['n_rows'])
train_df_after[2] = train_df_after[2].multiply(train_df_after['n_rows'])
train_df_after['n_rows'] = train_df_after['n_rows'].add(10)
train_df_after[0] = train_df_after[0].add(alfa * average_damage[0])
train_df_after[1] = train_df_after[1].add(alfa * average_damage[1])
train_df_after[2] = train_df_after[2].add(alfa * average_damage[2])
train_df_after[0] = train_df_after[0].divide(train_df_after['n_rows'])
train_df_after[1] = train_df_after[1].divide(train_df_after['n_rows'])
train_df_after[2] = train_df_after[2].divide(train_df_after['n_rows'])
train_df_after = train_df_after.drop(columns=['n_rows'])
    
global_mean = train_df_after.groupby('geo_level_2_id').mean()[[0,1,2]].reset_index()
test_df_after = test_df.reset_index().merge(global_mean,on='geo_level_2_id',how='left')
test_df_after = test_df_after.drop(columns=['geo_level_2_id']).set_index('building_id')
test_df_after[0] = test_df_after[0].fillna(average_damage[0])
test_df_after[1] = test_df_after[1].fillna(average_damage[1])
test_df_after[2] = test_df_after[2].fillna(average_damage[2])
train_df_after = train_df_after.drop(columns=['geo_level_2_id'])
train_df_after = train_df_after.sort_index()
target = target.sort_index()
test_df_after = test_df_after.sort_index()
test_target = test_target.sort_index()
train_df_after

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,geo_level_1_id_30,geo_level_1_id_4,geo_level_1_id_5,geo_level_1_id_6,geo_level_1_id_7,geo_level_1_id_8,geo_level_1_id_9,0,1,2
building_id,,,,,,,,,,,,,,,,,,,,,
4,1,25,5,2,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0.03,0.92,0.04
12,2,5,12,6,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0.01,0.10,0.89
16,2,80,5,4,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0.14,0.76,0.10
17,5,40,5,10,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0.04,0.70,0.26
31,2,30,10,5,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0.03,0.82,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052911,3,0,6,6,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0.02,0.52,0.47
1052921,3,25,9,7,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0.01,0.75,0.24
1052926,2,5,7,5,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0.35,0.60,0.06


In [9]:
cardinal_train = train_df.copy()
cardinal_test = test_df.copy()
train_df = train_df.drop(columns=['geo_level_2_id'])
test_df = test_df.drop(columns=['geo_level_2_id'])

In [10]:
dataset = tf.data.Dataset.from_tensor_slices((train_df.values, target.values))
dataset_after = tf.data.Dataset.from_tensor_slices((train_df_after.values, target.values))
for feat, targ in dataset_after.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [ 1.         25.          5.          2.          0.          1.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          1.
  0.          0.          1.          0.          0.          1.
  0.          0.          1.          0.          0.          0.
  0.          1.          0.          0.          0.          0.
  0.          1.          0.          0.          0.          1.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          1.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          1.
  0.          0

In [11]:
def compile_model(hp):
    layers = []
    l_amount = hp.Int('l_amount', min_value=3, max_value=9, step=1)
    l_size = hp.Int('l_size', min_value=200, max_value=1000, step=100)
    for x in range(l_amount):
        layers.append(tf.keras.layers.Dense(l_size, activation='relu'))
    layers.append(tf.keras.layers.Dense(units=3, activation='softmax'))
    model = tf.keras.Sequential(layers)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    opt = tf.keras.optimizers.Adam(clipnorm=1.0)
    model.compile(optimizer=opt,
                  loss=loss_fn,
                  metrics=['accuracy'])
    return model

In [12]:
tuner = kt.BayesianOptimization(compile_model,
                                objective='val_loss',
                                max_trials=25,
                                overwrite=True
                               )


In [13]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

tuner.search(
    train_df_after.to_numpy(), 
    target.to_numpy(), 
    epochs=25, 
    validation_data=(test_df_after, test_target),
    batch_size=128,
    callbacks=[stop_early],
)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


Trial 25 Complete [00h 05m 25s]
val_loss: 0.618537187576294

Best val_loss So Far: 0.6166689991950989
Total elapsed time: 02h 21m 49s
INFO:tensorflow:Oracle triggered exit


In [14]:
best_hps.get('l_size')

900

In [15]:
best_hps.get('l_amount')

8

In [16]:
save_best = tf.keras.callbacks.ModelCheckpoint('modelos/NNModelBY_Best', save_best_only=True)

best_model = compile_model(best_hps)
best_model.fit(dataset_after.batch(128), epochs=200, validation_data=(test_df_after, test_target), callbacks=[save_best])

Epoch 1/200
1018/1018 [==============================] - 24s 23ms/step - loss: 0.6845 - accuracy: 0.6883 - val_loss: 0.6420 - val_accuracy: 0.7056
INFO:tensorflow:Assets written to: modelos\NNModelBY_Best\assets
Epoch 2/200
1018/1018 [==============================] - 23s 22ms/step - loss: 0.6431 - accuracy: 0.7095 - val_loss: 0.6340 - val_accuracy: 0.7058
INFO:tensorflow:Assets written to: modelos\NNModelBY_Best\assets
Epoch 3/200
1018/1018 [==============================] - 23s 22ms/step - loss: 0.6348 - accuracy: 0.7124 - val_loss: 0.6317 - val_accuracy: 0.7071
INFO:tensorflow:Assets written to: modelos\NNModelBY_Best\assets
Epoch 4/200
1018/1018 [==============================] - 23s 22ms/step - loss: 0.6309 - accuracy: 0.7146 - val_loss: 0.6282 - val_accuracy: 0.7143
INFO:tensorflow:Assets written to: modelos\NNModelBY_Best\assets
Epoch 5/200
1018/1018 [==============================] - 23s 22ms/step - loss: 0.6272 - accuracy: 0.7167 - val_loss: 0.6284 - val_accuracy: 0.7138
Epoch

1018/1018 [==============================] - 22s 22ms/step - loss: 0.3385 - accuracy: 0.8531 - val_loss: 1.5012 - val_accuracy: 0.6752
Epoch 55/200
1018/1018 [==============================] - 23s 22ms/step - loss: 0.3357 - accuracy: 0.8550 - val_loss: 1.2965 - val_accuracy: 0.6735
Epoch 56/200
1018/1018 [==============================] - 23s 22ms/step - loss: 0.3278 - accuracy: 0.8586 - val_loss: 1.4685 - val_accuracy: 0.6726
Epoch 57/200
1018/1018 [==============================] - 23s 22ms/step - loss: 0.3225 - accuracy: 0.8609 - val_loss: 1.4643 - val_accuracy: 0.6782
Epoch 58/200
1018/1018 [==============================] - 22s 22ms/step - loss: 0.3220 - accuracy: 0.8613 - val_loss: 1.5332 - val_accuracy: 0.6795
Epoch 59/200
1018/1018 [==============================] - 23s 22ms/step - loss: 0.3165 - accuracy: 0.8639 - val_loss: 1.4237 - val_accuracy: 0.6716
Epoch 60/200
1018/1018 [==============================] - 22s 22ms/step - loss: 0.3158 - accuracy: 0.8650 - val_loss: 1.5935 

1018/1018 [==============================] - 22s 22ms/step - loss: 0.1706 - accuracy: 0.9334 - val_loss: 2.1198 - val_accuracy: 0.6601
Epoch 110/200
1018/1018 [==============================] - 23s 22ms/step - loss: 0.1712 - accuracy: 0.9331 - val_loss: 1.7435 - val_accuracy: 0.6621
Epoch 111/200
1018/1018 [==============================] - 22s 22ms/step - loss: 0.2903 - accuracy: 0.9344 - val_loss: 2.1222 - val_accuracy: 0.6628
Epoch 112/200
1018/1018 [==============================] - 23s 22ms/step - loss: 0.1641 - accuracy: 0.9358 - val_loss: 1.8806 - val_accuracy: 0.6645
Epoch 113/200
1018/1018 [==============================] - 23s 22ms/step - loss: 0.1608 - accuracy: 0.9373 - val_loss: 2.0853 - val_accuracy: 0.6561
Epoch 114/200
1018/1018 [==============================] - 23s 22ms/step - loss: 0.1629 - accuracy: 0.9371 - val_loss: 3.4151 - val_accuracy: 0.6673
Epoch 115/200
1018/1018 [==============================] - 23s 22ms/step - loss: 0.1629 - accuracy: 0.9373 - val_loss: 2

1018/1018 [==============================] - 24s 23ms/step - loss: 0.1020 - accuracy: 0.9650 - val_loss: 2.0652 - val_accuracy: 0.6565
Epoch 165/200
1018/1018 [==============================] - 23s 23ms/step - loss: 0.0925 - accuracy: 0.9666 - val_loss: 2.2174 - val_accuracy: 0.6553
Epoch 166/200
1018/1018 [==============================] - 23s 23ms/step - loss: 0.0937 - accuracy: 0.9677 - val_loss: 2.0454 - val_accuracy: 0.6542
Epoch 167/200
1018/1018 [==============================] - 23s 22ms/step - loss: 0.0901 - accuracy: 0.9680 - val_loss: 2.0720 - val_accuracy: 0.6552
Epoch 168/200
1018/1018 [==============================] - 22s 22ms/step - loss: 0.0931 - accuracy: 0.9671 - val_loss: 2.1850 - val_accuracy: 0.6604
Epoch 169/200
1018/1018 [==============================] - 23s 22ms/step - loss: 0.0877 - accuracy: 0.9700 - val_loss: 2.3432 - val_accuracy: 0.6589
Epoch 170/200
1018/1018 [==============================] - 22s 22ms/step - loss: 0.0907 - accuracy: 0.9687 - val_loss: 2

In [17]:
best_model.save('modelos/NNModelBY')

INFO:tensorflow:Assets written to: modelos/NNModelBY\assets


In [18]:
tf.keras.models.load_model('modelos/NNModelBY_Best').evaluate(test_df_after, test_target)

4072/4072 [==============================] - 13s 3ms/step - loss: 0.6271 - accuracy: 0.7160


[0.6270622611045837, 0.7160497903823853]

In [22]:
global_mean.set_index('geo_level_2_id').to_csv('geolevel_2_id_mean.csv')

In [21]:
average_damage

array([0.09630852, 0.56933998, 0.3343515 ])